In [ ]:
filename = 'input.txt'

with open(filename) as f:
    stones = f.readlines()
    stones = stones[0].split(' ')
    stones = [x.split('\n')[0] if '\n' in x else x for x in stones] 
    stones = [int(x) for x in stones]
    
stones
    

In [ ]:
def evolve(number, level, stop_level):
    if level == stop_level:
        return 1, {number: 1}
    elif number == 0:
        return evolve(1, level+1, stop_level)
    elif len(str(number)) % 2 == 0:
        num_list = list(str(number))
        stop = int(len(num_list)/2)
        first_half = num_list[:stop]
        second_half = num_list[stop:]
        amount_from_first, first_dict = evolve(int("".join(first_half)), level+1, stop_level) 
        amount_from_second, second_dict = evolve(int("".join(second_half)), level+1, stop_level)
        for key in first_dict:
            if not key in second_dict:
                second_dict[key] = 0
            second_dict[key] = second_dict[key] + first_dict[key]
        return (amount_from_first + amount_from_second), second_dict
    else:
        return evolve(number * 2024, level + 1, stop_level)

In [ ]:
total = 0    
master_dict = {}

for num in stones:
    this_total, this_dict = evolve(num, 0, 25)
    total += this_total
    for key in this_dict:
        if key not in master_dict:
            master_dict[key] = 0
        master_dict[key] = master_dict[key] + this_dict[key]

total

In [ ]:
total_2 = 0    
master_dict_2 = {}

for master_key in master_dict:
    this_total, this_dict = evolve(master_key, 0, 25)
    total_2 += this_total * master_dict[master_key]
    for key in this_dict:
        if key not in master_dict_2:
            master_dict_2[key] = 0
        master_dict_2[key] = master_dict_2[key] + this_dict[key] * master_dict[master_key]

total_2

In [ ]:
master_dict_2

In [ ]:
total_3 = 0    
master_dict_3 = {}

for master_key in master_dict_2:
    this_total, this_dict = evolve(master_key, 0, 25)
    total_3 += this_total * master_dict_2[master_key]
    for key in this_dict:
        if key not in master_dict_3:
            master_dict_3[key] = 0
        master_dict_3[key] = master_dict_3[key] + this_dict[key] * master_dict_2[master_key]

total_3

In [ ]:
def thread_evolve(num, lev, stop_lev, q):
    def evolve(number, level, stop_level):
        if level == stop_level:
            return 1, {number: 1}
        elif number == 0:
            return evolve(1, level+1, stop_level)
        elif len(str(number)) % 2 == 0:
            num_list = list(str(number))
            stop = int(len(num_list)/2)
            first_half = num_list[:stop]
            second_half = num_list[stop:]
            amount_from_first, first_dict = evolve(int("".join(first_half)), level+1, stop_level) 
            amount_from_second, second_dict = evolve(int("".join(second_half)), level+1, stop_level)
            for key in first_dict:
                if not key in second_dict:
                    second_dict[key] = 0
                second_dict[key] = second_dict[key] + first_dict[key]
            return (amount_from_first + amount_from_second), second_dict
        else:
            return evolve(number * 2024, level + 1, stop_level)
        
    result_sum, _ = evolve(num, lev, stop_lev)
    q.put(result_sum)

In [ ]:
import queue
from threading import Thread

total_3 = 0    
master_dict_3 = {}

q = queue.Queue()
threads = []

for master_key in master_dict_2:
    t = Thread(target=thread_evolve, args=(master_key, 0, 25, q))
    t.start()
    threads.append(t)

for t in threads:
    t.join()

[q.get() for _ in range(len(q))]